In [4]:
%load_ext autoreload
%autoreload 2

import ee
import folium
import utils
# Authenticate to Earth Engine
ee.Authenticate()
# Initialize Earth Engine instance
ee.Initialize(project="reeftruth")
#!!! (for readme)
# https://stackoverflow.com/questions/78374548/no-authentication-box-appears-when-authenticating-google-earth-engine-gee-pyth


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define functions

In [5]:
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = utils.add_ee_layer

### Set visualisation parameters

In [6]:
protectedareas_vps = {
  "palette": ["2ed033"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.05
}
gdcr_vps = {
  "palette": ["green"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.8
}
wri_vps = {
  "palette": ["red"],
  "min": 0.0,
  "max": 1550000.0,
  "opacity": 0.4
}
reefcheck_vps = {
  "color": "white",  
  "pointSize": 200   
}

copernicus_vps = {
  "min": 0,
  "max": 2,
  "gamma": 1.5
}

# centring and zoom
lat = -20
lon = 150
zoom = 10

## Add reef layers to map

TODO: detail layers

In [7]:

# initialise map with bacgkround image
reef_map = folium.Map(location=[lat, lon], zoom_start=zoom)

### COPERNICUS
# Load Copernicus image collection and filter by date
copernicus = ee.ImageCollection("COPERNICUS/S3/OLCI").filterDate("2018-02-02", "2018-02-03")

# Select bands for visualization and apply band-specific scale factors.
copernicus_image = copernicus.select(["Oa08_radiance", "Oa06_radiance", "Oa04_radiance"]) \
                              .median() \
                              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))

reef_map.add_ee_layer(copernicus_image, copernicus_vps, "copernicus background")

### ALLEN CORAL ATLAS
aca_ds = ee.Image("ACA/reef_habitat/v2_0") 

# Example mask application
aca_reef_extent = aca_ds.select("reef_mask").selfMask()
reef_map.add_ee_layer(aca_reef_extent, {"palette": ["EEEEEE"]}, "ACA Global reef extent")

# Geomorphic zonation classification
aca_geomorphic_zonation = aca_ds.select("geomorphic").selfMask()
reef_map.add_ee_layer(aca_geomorphic_zonation, {}, "ACA Geomorphic zonation")

# Benthic habitat classification
aca_benthic_habitat = aca_ds.select("benthic").selfMask()
reef_map.add_ee_layer(aca_benthic_habitat, {"palette": ["FF7F50"]}, "ACA Benthic habitat")


### UNEP-WCMC World Database of Coral Reefs (GDCR)
unep_gdcr = ee.FeatureCollection("projects/reeftruth/assets/WCMC008_CoralReef2021_Py_v4_1") 
gdcr_image = ee.Image().float().paint(unep_gdcr, "GDCR")
reef_map.add_ee_layer(gdcr_image, gdcr_vps, "UNEP-GDCR_polygons")

### World Resources Institute (WRI) Reefs at Risk
wri_500 = ee.FeatureCollection("projects/reeftruth/assets/reef_500_poly") 
wri_image = ee.Image().float().paint(wri_500, "WRI_500")
reef_map.add_ee_layer(wri_image, wri_vps, "WRI_500_polygons")

### Reef Check
# # reef_check = ee.FeatureCollection("projects/...") 
# # reef_map.add_child(reef_check, reefcheck_vps, "Reef Check points")

### World Database on Protected Areas (WDPA)
wdpa_ds = ee.FeatureCollection("WCMC/WDPA/current/polygons")
wdpa_image = ee.Image().float().paint(wdpa_ds, "REP_AREA")
reef_map.add_ee_layer(wdpa_image, protectedareas_vps, "protected areas")

reef_map.add_child(folium.LayerControl())
display(reef_map)

In [8]:
ee.List([wdpa_ds.filter(ee.Filter.eq("geotype", "Polygon").geometry())])

TypeError: Filter.geometry() missing 1 required positional argument: 'geometry'

In [27]:
# get basic info about a feature collection

def get_info(fc):
    """Can't just call .getInfo() on very large feature collections. This breaks it down"""
    print(f"Number of elements: {fc.size().getInfo()}")
    print(f"Type of first element: {fc.first().geometry().type().getInfo()}")
    print(f"Approximate size of first element: {fc.first().geometry().area().divide(1000000).getInfo()} km$^2$")
    return fc.getInfo()

def calculate_polygon_area(poly):
    """Calculate the area of a polygon in km^2"""
    return poly.area().divide(1000000).getInfo()

def calculate_feature_collection_area(fc):
    """Calculate the total area of a feature collection in km^2"""
    for poly in fc:
        print(calculate_polygon_area(poly))
    # return fc.geometry().area().divide(1000000).getInfo()

In [32]:
study_region = ee.Geometry.Rectangle(130, -32, 170, 0)
# subset fc
gbr_wri_500 = wri_500.filterBounds(study_region)
gbr_wdpa_ds = wdpa_ds.filterBounds(study_region)


In [35]:
wri_500.size().getInfo()

1

In [45]:
geoms = wri_500.geometry().geometries()
features = ee.FeatureCollection(geoms.map(lambda geo: ee.Feature(geo)))

In [47]:
features.first().getInfo()

EEException: Collection: Geometries cannot have their properties modified or be placed into collections.

In [40]:
wri_500.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[175.03575393732905, -1.5094276710920536],
     [175.04024429031924, -1.5094277080401812],
     [175.04024429031924, -1.49586141461354],
     [175.03575393732905, -1.4958613930272284],
     [175.03575393732905, -1.5094276710920536]]],
   [[[175.02677333843815, -1.473242149479401],
     [175.02227857262716, -1.473242165259142],
     [175.02227857262716, -1.477768714599051],
     [175.02677333843815, -1.4777687286207992],
     [175.02677333843815, -1.473242149479401]]],
   [[[175.01778828352917, -1.4913348934731296],
     [175.02227857262716, -1.4913348488556506],
     [175.02227857262716, -1.4868128141218577],
     [175.01778828352917, -1.4868128149451805],
     [175.01778828352917, -1.4913348934731296]]],
   [[[175.01329795141095, -1.4777686789224107],
     [175.01778828352917, -1.4777687014128507],
     [175.01778828352917, -1.4732421818714785],
     [175.01329795141095, -1.473242189207651],
     [175.013297

In [43]:
wri_500.type().getInfo()

AttributeError: 'FeatureCollection' object has no attribute 'type'

In [39]:
wri_500.getInfo()

{'type': 'FeatureCollection',
 'columns': {'GRIDCODE': 'Long', 'system:index': 'String'},
 'version': 1714316029362563,
 'id': 'projects/reeftruth/assets/reef_500_poly',
 'properties': {'system:asset_size': 6859883},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[175.03575393732905, -1.5094276710920536],
       [175.04024429031924, -1.5094277080401812],
       [175.04024429031924, -1.49586141461354],
       [175.03575393732905, -1.4958613930272284],
       [175.03575393732905, -1.5094276710920536]]],
     [[[175.02677333843815, -1.473242149479401],
       [175.02227857262716, -1.473242165259142],
       [175.02227857262716, -1.477768714599051],
       [175.02677333843815, -1.4777687286207992],
       [175.02677333843815, -1.473242149479401]]],
     [[[175.01778828352917, -1.4913348934731296],
       [175.02227857262716, -1.4913348488556506],
       [175.02227857262716, -1.4868128141218577],
       [175.01778828352917, -1.486812814945180

In [31]:
wdpa_ds.size().getInfo()

283531

In [33]:
gbr_wdpa_ds.size().getInfo()

2615

In [26]:
wdpa_ds.first().geometry().area().divide(1000000).getInfo()

447888.34210428083

In [20]:
# wdpa_ds.union().geometry().getInfo()
wri_500.first().getInfo()

KeyboardInterrupt: 